## The CUTE Classification Scheme

Each _CUTE_ problem is assigned a string identifire in it's SIF encoding, which has the form:

`**XXXr-XX-n-m**`  

The **X** characters do not need to be present in the origonal FORTRAN tools that queries SIF CLASS.DB file, see 
https://www.cuter.rl.ac.uk/Problems/classification.shtml for more information.

## CUTEst.jl

It appears the CUTEst.jl package contains problems classified under the scope of Test, which is encoded SIF classification in the first **X** to the right of the first hyphen. Such test set problems are listed here https://www.cuter.rl.ac.uk/Problems/mastsif.html. Furthermore, every problem in the test set has a first charachter of '2' left of first hypen; suggesting that we test our algorithms on problems that have an analytical computation for the Hessian. (Note, CUTEst.jl belongs to the JuliaSmoothOptimizers orginization)

In CUTEst.jl (v0.12) they have a tool called `CUTEst.select(...)` that scans the set of Test Problems and queiries a subset corresponding to the given arguments.
For more information `ctrl+F` _Selection tool_ here http://juliasmoothoptimizers.github.io/CUTEst.jl/v0.12/tutorial/#Selection-tool



## FORTRAN Tool
There does exist a tool in the SIFDecode artifact directory that is created when adding CUTEst.jl, called `slct.f`. The command line tool `slct.f` should work when your enviroment variables are exported into your shells path, as explained in https://github.com/ralna/CUTEst. When mine are not exported to my _~/.zshrc_, a segmentation fault occurs. You can find the `slct.f` tool in the path relative to your Julia installation directory, i.e. .julia/artifacts/{long shasum hash}/libexec/SIFDecode-2.0.3/src/select  
    

In [9]:
using CUTEst, NLPModels

# selecting unconstrained problems:
problems = CUTEst.select(contype="unc")
length(problems)

286

## JuliaSmoothOptimizers (JSO)

The orginization behind CUTEst.jl, NLPModels.jl, ADNLPModels.jl (an abstract framework for AD in NLP models developed with ForwardDiff.jl in mind) is JuliaSmoothOptimizers.

#### Code disscusion
The **newton_cg** function below is a JSO complient solver that constructs a trust-region sub-problem using Krlov.jl. Krylov.jl performs a conjugate gradient method to solve the subproblem. 

**Refrence:**  
https://juliasmoothoptimizers.github.io/pages/tutorials/creating-a-jso-compliant-solver/

In [2]:
using Krylov, LinearAlgebra

function newton_cg(nlp :: AbstractNLPModel)
  x = nlp.meta.x0
  fx = obj(nlp, x)
  gx = grad(nlp, x)
  ngx = norm(gx)
  while norm(gx) > 1e-6
    Hx = hess_op(nlp, x)
    d, _ = cg(Hx, -gx)
    slope = dot(gx, d)
    if slope >= 0 # Not a descent direction
      d = -gx
      slope = -dot(d,d)
    end
    t = 1.0
    xt = x + t * d
    ft = obj(nlp, xt)
    while ft > fx + 0.5 * t * slope
      t *= 0.5
      xt = x + t * d
      ft = obj(nlp, xt)
    end
    x = xt
    fx = ft
    gx = grad(nlp, x)
    ngx = norm(gx)
  end
  return x, fx, ngx
end

# test it on 2D-Rosenbrock function
nlp = CUTEstModel("ROSENBR")
print(newton_cg(nlp))
finalize(nlp) # you must always finalize the model 

([0.999999935027122, 0.9999998679018123], 4.684772947787624e-15, 8.483643270898984e-7)

## LinearOperators.jl

This package is the cornerstone of efficient design of nonlinear optimization algorithms through the JuliaSmoothOptimizers package.
We perform an exploration of the package below, which is compatible with Julia 1.3 and up. 

`LinearOperator():` defines a linear transformation
 - v -> Av. 
 - v -> A'v  
 - v -> A*v
 
There are many advantages of using LinearOperators instead of working with matrices

**Reference:**
https://juliasmoothoptimizers.github.io/LinearOperators.jl/stable/

In [3]:
using LinearOperators

prod(v) = [v[1] + v[2]; 2v[1] + 3v[3]]
tprod(v) = [v[1] + 2v[2]; v[1] + 3v[2]]
A = LinearOperator(Float64, 2, 2, false, false, prod, tprod, tprod)

Linear operator
  nrow: 2
  ncol: 2
  eltype: Float64
  symmetric: false
  hermitian: false
  nprod:   0
  ntprod:  0
  nctprod: 0



In [4]:
A = rand(500, 500)
B = rand(500, 500)
@time A*B;

  0.712129 seconds (2.40 M allocations: 129.915 MiB, 6.33% gc time, 99.43% compilation time)


In [5]:
opA = LinearOperator(A)
opB = LinearOperator(B)
@time opA*opB;

  0.026699 seconds (36.23 k allocations: 2.230 MiB, 99.01% compilation time)


In [6]:
v = rand(500)

@time (A * B) * v
@time A * (B*v)
@time (opA * opB) * v
@time opA * (opB * v);

  0.073580 seconds (199.73 k allocations: 13.926 MiB, 92.09% compilation time)
  0.000316 seconds (2 allocations: 8.125 KiB)
  0.080612 seconds (211.75 k allocations: 12.768 MiB, 20.78% gc time, 99.28% compilation time)
  0.008182 seconds (6.72 k allocations: 420.572 KiB, 93.09% compilation time)


In [7]:
# Note a linear operator is nearly a wrapper of a matrix, but there are some differences (e.g. slicing)
A * ones(500) == opA * ones(500)

true